# Гипотеза о `font_size`

Предполагается следующее: ширина текста пропорциональна `font_size`.

In [1]:
import numpy as np
import pandas as pd

from lets_plot import *
from lets_plot.mapping import as_discrete
LetsPlot.setup_html()

import os; import sys; sys.path.append(os.path.join(sys.path[0], ".."))
from util import util

In [2]:
def add_basic_width(df, basic_size):
    def calc_basic_width(r):
        basic_width = int(r[r.font_size == basic_size].iloc[0].width)
        r["basic_width"] = pd.Series(basic_width, index=r.index)
        return r

    df["basic_width"] = np.nan
    return df.groupby("char").apply(calc_basic_width)

In [3]:
BASIC_FONT_SIZE = 14
BASIC_FONT = util.Font("Lucida Grande", BASIC_FONT_SIZE, "italic")

In [4]:
char_widths_df = util.get_df("../data/full/char_widths.csv", "all")
char_widths_df = util.filter_by_font(char_widths_df, BASIC_FONT, filters=["family", "face"])
char_widths_df

,char_id,char,alphabet,font_size,width
0,65,A,basic_latin,9,8
1,66,B,basic_latin,9,8
2,67,C,basic_latin,9,10
3,68,D,basic_latin,9,9
4,69,E,basic_latin,9,9
...,...,...,...,...,...
1207,123,{,russian,20,13
1208,124,|,russian,20,7
1209,125,},russian,20,11
1210,126,~,russian,20,16


In [5]:
font_sizes = list(char_widths_df.font_size.unique())
font_sizes

[9, 11, 12, 14, 16, 20]

In [6]:
char_widths_df = add_basic_width(char_widths_df, basic_size=BASIC_FONT_SIZE)
char_widths_df["width_coeff"] = char_widths_df.width / char_widths_df.basic_width
char_widths_df["height_coeff"] = char_widths_df.font_size / BASIC_FONT_SIZE
char_widths_df["stretch_coeff"] = char_widths_df.width_coeff / char_widths_df.height_coeff
char_widths_df.head()

,char_id,char,alphabet,font_size,width,basic_width,width_coeff,height_coeff,stretch_coeff
0,65,A,basic_latin,9,8,14,0.571429,0.642857,0.888889
1,66,B,basic_latin,9,8,13,0.615385,0.642857,0.957265
2,67,C,basic_latin,9,10,15,0.666667,0.642857,1.037037
3,68,D,basic_latin,9,9,14,0.642857,0.642857,1.000000
4,69,E,basic_latin,9,9,14,0.642857,0.642857,1.000000


Пусть для символа `X` и размера шрифта `S` величина `w(X, S)` означает ширину символа в пикселях.

**Гипотеза A:**
Если величина `'stretch_coeff'` близка к константе `C`, то зная величину `w(X, B)` для базового размера `B`, величину для размера `S` можно вычислить по формуле `w(X, S) = C * (S / B) * w(X, B)`. Т.е. для пересчета модели от базового размера `B` к некоторому новому размеру `S`, достаточно знать одну заранее вычисленную константу.

**Гипотеза B:**
Если `'stretch_coeff'` не константна, но хотя бы величины `'width_coeff'` близки к константе `C(S)` для каждого зафиксированного размера `S`, то величину `w(X, S)` можно вычислять по крайней мере по формуле `w(X, S) = C(S) * w(X, B)`. Т.е. для пересчета модели от базового размера `B` к некоторому новому размеру `S`, нужно для каждого интересующего нас размера `S` заранее вычислить соответствующую ему константу.

Если даже `'width_coeff'` не ведут себя как константы для каждого `S`, то это означает, что при изменении размера шрифта, ширина ведет себя более сложным образом, чем простое растяжение.

## Проверка гипотезы A

In [7]:
ggplot(char_widths_df) + \
    geom_density(aes(x="stretch_coeff"), color="black", fill="#8da0cb") + \
    ggtitle("Вариация 'stretch_coeff'")

In [8]:
ggplot(char_widths_df) + \
    geom_boxplot(aes("alphabet", "stretch_coeff", fill="alphabet")) + \
    ggtitle("Диапазон изменения 'stretch_coeff' для каждого алфавита")

In [9]:
plot_df = char_widths_df.groupby(["alphabet", "char"]).stretch_coeff.agg(["min", "max", "median"]).reset_index()
ggplot(plot_df) + \
    geom_pointrange(aes(x=as_discrete("char", order_by="median", order=1), \
                        y="median", ymin="min", ymax="max", color="alphabet")) + \
    facet_grid(y="alphabet") + \
    ylab("stretch_coeff") + \
    ggtitle("Изменение 'stretch_coeff' посимвольно") + \
    ggsize(800, 400) + \
    theme_minimal() + theme(axis_text_x="blank")

Очевидно, что `'stretch_coeff'` совсем не константа. Однако, диапазон изменения достаточно маленький. Как понять, это много или мало для прогнозирования? Попробуем делать предсказания с помощью медианного значения `'stretch_coeff'`.

In [10]:
stretch_coeff = char_widths_df.stretch_coeff.median()
stretch_coeff

0.9898989898989898

In [11]:
char_widths_df["stretched_width"] = np.round(stretch_coeff * char_widths_df.font_size / BASIC_FONT_SIZE * char_widths_df.basic_width).astype(int)
char_widths_df["stretched_width_error"] = char_widths_df.stretched_width - char_widths_df.width
char_widths_df.head()

,char_id,char,alphabet,font_size,width,basic_width,width_coeff,height_coeff,stretch_coeff,stretched_width,stretched_width_error
0,65,A,basic_latin,9,8,14,0.571429,0.642857,0.888889,9,1
1,66,B,basic_latin,9,8,13,0.615385,0.642857,0.957265,8,0
2,67,C,basic_latin,9,10,15,0.666667,0.642857,1.037037,10,0
3,68,D,basic_latin,9,9,14,0.642857,0.642857,1.000000,9,0
4,69,E,basic_latin,9,9,14,0.642857,0.642857,1.000000,9,0


In [12]:
ggplot(char_widths_df) + \
    geom_bar(aes(x="stretched_width_error", fill="font_size"), color="black", \
             tooltips=layer_tooltips().format("@font_size", 'd').title("Font size: @font_size")\
                                      .format("@stretched_width_error", 'd').line("error|@stretched_width_error")\
                                      .format("@..count..", 'd').line("chars number|@..count..")\
                                      .line("alphabet|@alphabet")) + \
    scale_fill_brewer(type="seq", palette="Oranges", direction=-1, breaks=font_sizes) + \
    facet_grid(x="alphabet", y="font_size")

Из графиков видно, что:

- Ошибки встречаются. Иногда больше чем на пиксель. И они будут накапливаться с увеличением длины текста.

- Чаще идет ошибка в большую сторону, т.е. ширина будет скорее завышаться.

- Когда шрифт больше базового, ошибок больше, чем когда он меньше. Возможно, это означает, что все-таки правильнее брать в качестве базового шрифт бóльшего размера.

## Проверка гипотезы B

In [13]:
ggplot(char_widths_df) + \
    geom_violin(aes(as_discrete("font_size", order=1), "width_coeff"), fill="#8da0cb") + \
    ggtitle("Вариация 'width_coeff'")

`'width_coeff'` ведет себя подозрительно для базового размера шрифта, но это объясняется тем, что именно в этом случае величина действительно константна:

In [14]:
char_widths_df[char_widths_df.font_size == BASIC_FONT_SIZE].describe().width_coeff

count    202.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: width_coeff, dtype: float64

In [15]:
ggplot(char_widths_df) + \
    geom_boxplot(aes(as_discrete("font_size", order=1), "width_coeff", fill="alphabet")) + \
    facet_grid(x="alphabet") + \
    coord_flip() + \
    ggtitle("Диапазон изменения 'width_coeff' для каждого алфавита и размера шрифта") + \
    theme_minimal()

Вариация маленькая, но она все еще есть.

Снова попробуем делать предсказания с помощью медианного значения `'width_coeff'` для каждого размера, чтобы посмотреть как на этот раз будет выглядеть ошибка.

In [16]:
width_coeffs = {
    font_size: char_widths_df[char_widths_df.font_size == font_size].width_coeff.median()
    for font_size in font_sizes
}
width_coeffs

{9: 0.6363636363636364,
 11: 0.7857142857142857,
 12: 0.8333333333333334,
 14: 1.0,
 16: 1.0909090909090908,
 20: 1.3846153846153846}

In [17]:
char_widths_df["restored_width"] = np.round(char_widths_df.font_size.replace(width_coeffs) * char_widths_df.basic_width).astype(int)
char_widths_df["restored_width_error"] = char_widths_df.restored_width - char_widths_df.width
char_widths_df.head()

,char_id,char,alphabet,font_size,width,basic_width,width_coeff,height_coeff,stretch_coeff,stretched_width,stretched_width_error,restored_width,restored_width_error
0,65,A,basic_latin,9,8,14,0.571429,0.642857,0.888889,9,1,9,1
1,66,B,basic_latin,9,8,13,0.615385,0.642857,0.957265,8,0,8,0
2,67,C,basic_latin,9,10,15,0.666667,0.642857,1.037037,10,0,10,0
3,68,D,basic_latin,9,9,14,0.642857,0.642857,1.000000,9,0,9,0
4,69,E,basic_latin,9,9,14,0.642857,0.642857,1.000000,9,0,9,0


In [18]:
ggplot(char_widths_df) + \
    geom_bar(aes(x="restored_width_error", fill="font_size"), color="black", \
             tooltips=layer_tooltips().format("@font_size", 'd').title("Font size: @font_size")\
                                      .format("@restored_width_error", 'd').line("error|@restored_width_error")\
                                      .format("@..count..", 'd').line("chars number|@..count..")\
                                      .line("alphabet|@alphabet")) + \
    scale_fill_brewer(type="seq", palette="Oranges", direction=-1, breaks=font_sizes) + \
    facet_grid(x="alphabet", y="font_size")

Из новых графиков видно, что:

- Ошибки встречаются, как и в прошлый раз, хотя ожидалось, что такая модель будет точнее.

- Ошибка выглядит распределенной более равномерно. В том смысле, что по сравнению с предыдущим аналогичным графиком, символов с отрицательным отклонением примерно столько же, сколько символов с положительным, что может положительно повлиять на общий прогноз, когда текст достаточно длинный.

- Вновь, когда шрифт больше базового, ошибок больше, чем когда он меньше.

Хотелось бы более точно сравнить две получившиеся модели через ошибки в их прогнозах, поэтому построим еще один график:

In [19]:
ggplot(char_widths_df) + \
    geom_bar(aes(x="stretched_width_error"), \
             width=.4, position=position_nudge(x=-.2), \
             color="black", fill="#66c2a5", \
             tooltips=layer_tooltips().title("'stretched_width_error'")\
                                      .format("@stretched_width_error", 'd').line("error|@stretched_width_error")\
                                      .format("@..count..", 'd').line("chars number|@..count..")) + \
    geom_bar(aes(x="restored_width_error"), \
             width=.4, position=position_nudge(x=.2), \
             color="black", fill="#fc8d62", \
             tooltips=layer_tooltips().title("'restored_width_error'")\
                                      .format("@restored_width_error", 'd').line("error|@restored_width_error")\
                                      .format("@..count..", 'd').line("chars number|@..count.."))

Видно, что:

- Вторая модель в целом дает чуть меньше ошибок.

- Для второй модели действительно характерно более равномерное распределение ошибок: она примерно одинаково часто ошибается в большую и в меньшую сторону. По крайней мере, по сравнению с первой моделью, с единственным коэффициентом растяжения - `'stretch_coeff'`.

- На самом деле, разница между моделями, как будто бы, не очень большая.

## Заключение

На практике, скорее всего, оба варианта должны дать достаточно неплохой прогноз, но это желательно проверить на датасете с полноценными текстами. Если ошибка в обоих случаях будет удовлетворительной, то, конечно, более предпочтительной будет модель с единственным коэффициентом для перехода между шрифтами разного размера.

Также надо учитывать, что при проверке гипотез **A** и **B** предполагалось, что у нас есть модель, идеально вычисляющая ширину символов для базового шрифта. На самом деле, на практике будет более грубая модель. Для этой модели конкретные коэффициенты понадобится пересчитать.

Наконец, можно проверить те же самые гипотезы для другого способа вычисления коэффициентов. Их можно вычислять не на основе отдельных символов, а на основе небольших текстов, т.е. во всех предыдущих ячейках использовать другой датафрейм, в котором вместо `'char'` будет использоваться столбец `'text'`. Это повлияет на значение коэффициентов `'stretch_coeff'` и `'width_coeff'`, но в лучшую ли сторону - пока не очевидно.